In [1]:
from tqdm.notebook import tqdm
import qutip as qt
import numpy as np
from importlib import reload
import scipy.sparse

import qims as qims
from matplotlib import pyplot as plt


In [49]:
Nx= 26;
%time test=[r for r in range(2 ** Nx)]

CPU times: user 2.84 s, sys: 874 ms, total: 3.72 s
Wall time: 3.71 s


In [107]:
def func1(x, Z): return Z

# %time a, b, c = qt.parfor(func1, range(10),Z=20)
qt.parallel_map(func1, range(10), task_kwargs={'Z':10},progress_bar=True)


10.0%. Run time:   0.23s. Est. time left: 00:00:00:02
20.0%. Run time:   0.24s. Est. time left: 00:00:00:00
30.0%. Run time:   0.24s. Est. time left: 00:00:00:00
40.0%. Run time:   0.24s. Est. time left: 00:00:00:00
50.0%. Run time:   0.24s. Est. time left: 00:00:00:00
60.0%. Run time:   0.24s. Est. time left: 00:00:00:00
70.0%. Run time:   0.24s. Est. time left: 00:00:00:00
80.0%. Run time:   0.24s. Est. time left: 00:00:00:00
90.0%. Run time:   0.24s. Est. time left: 00:00:00:00
100.0%. Run time:   0.24s. Est. time left: 00:00:00:00
Total run time:   0.34s


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [30]:
Nx = 6
Basis = []
def F(prms):

    st = qmb.ind2state(prms[0], prms[1])
    inds = np.dot(st, np.roll(st, 1))
    if inds == 0.0:
        return r
    else:
        return -1


%time test = qt.parfor(F, range(2 ** Nx))

CPU times: user 18.2 ms, sys: 59.3 ms, total: 77.5 ms
Wall time: 88.3 ms


In [34]:
test2 = [test[r] for r in range(len(test)) if test[r]>-0.5]
len(test2)

18

In [35]:
np.array(test2)

array([ 0,  1,  2,  4,  5,  8,  9, 10, 16, 17, 18, 20, 21, 32, 34, 36, 40,
       42])

In [37]:
np.array([bs[r] for r in range(len(bs))])

array([ 0,  1,  2,  4,  5,  8,  9, 10, 16, 17, 18, 20, 21, 32, 34, 36, 40,
       42])

In [31]:
%time bs, bs_ind = qmb.basis(Nx)

  0%|          | 0/64 [00:00<?, ?it/s]

CPU times: user 16.2 ms, sys: 9.8 ms, total: 26 ms
Wall time: 21.8 ms


# Translational invariance

(07/15/22)

In [4]:
reload(qmb)
Nx = 8
%time bs, bs_ind = qmb.basis(Nx)
%time H = qmb.pxp_hamiltonian(bs,bs_ind,Nx)
%time vbs = qmb.GenerateMomentumBasis(Nx,bs)

Trn = 0
for n in range(len(vbs)):
    inds = list(map(bs_ind.get, vbs[n]))
    row = inds
    col = np.roll(inds,1)
    data = np.ones(len(np.roll(inds,1)))

    Trn = Trn + scipy.sparse.csr_matrix((data, (row, col)), shape=(len(bs),\
                                                                  len(bs))).T
Trn = qt.Qobj(Trn)

  0%|          | 0/256 [00:00<?, ?it/s]

CPU times: user 31.8 ms, sys: 0 ns, total: 31.8 ms
Wall time: 27.6 ms


  0%|          | 0/8 [00:00<?, ?it/s]

CPU times: user 28.9 ms, sys: 3.42 ms, total: 32.3 ms
Wall time: 30.1 ms
CPU times: user 4.67 ms, sys: 132 µs, total: 4.8 ms
Wall time: 4.17 ms


In [5]:
np.sum(np.abs((Trn*H-H*Trn).full()))

0.0

# Check that full and momentum Hamiltonians produce the same spectra

In [7]:
k_list = np.arange(0,Nx)/Nx

scan = {}
for kk in k_list:
    scan[kk] = []

for n in tqdm(range(len(vbs))):

    ktemp_list = np.arange(0,len(vbs[n]))/len(vbs[n])
    inds = list(map(bs_ind.get, vbs[n]))

    for kk in ktemp_list:
        phs = np.exp(-2*np.pi*kk*1j*np.arange(0,len(inds)))/np.sqrt(len(inds))
        scan[kk].append([inds,phs])


U = {}
for kk in tqdm(k_list):
    # print('---------------------------')
    U[kk] = 0
    for n in range(len(scan[kk])):
        row = n*np.ones(len(scan[kk][n][0]))
        col = np.array(scan[kk][n][0])
        data = np.array(scan[kk][n][1])

        U[kk] = U[kk] + scipy.sparse.csr_matrix((data, (row, col)), shape=(len(scan[kk]),\
                                                                      len(bs))).T
        # print(qt.Qobj(U[kk]))


Hs = {}
scan = []
for k in tqdm(k_list):
    Hs[k]=(qt.Qobj(U[k]).dag()*H*qt.Qobj(U[k]))
    scan.append(Hs[k].eigenstates()[0])
scan = [x for xs in scan for x in xs]

de = H.eigenenergies()-np.sort(scan)
np.sum(np.abs(de))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

1.8567547639657043e-13

# Periodic and open boundary conditions

(07/15/22)

In [2]:
bs = qmb.basis(6, bc = "periodic")[0]
print(len(bs))
[qmb.ind2state(bs[r],6) for r in range(len(bs))]

  0%|          | 0/64 [00:00<?, ?it/s]

18


[array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 1, 0, 1]),
 array([0, 0, 1, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 1]),
 array([0, 0, 1, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 1]),
 array([0, 1, 0, 0, 1, 0]),
 array([0, 1, 0, 1, 0, 0]),
 array([0, 1, 0, 1, 0, 1]),
 array([1, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 1, 0]),
 array([1, 0, 0, 1, 0, 0]),
 array([1, 0, 1, 0, 0, 0]),
 array([1, 0, 1, 0, 1, 0])]

In [3]:
bs = qmb.basis(6, bc = "open")[0]
print(len(bs))
[qmb.ind2state(bs[r],6) for r in range(len(bs))]

  0%|          | 0/64 [00:00<?, ?it/s]

21


[array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 1, 0, 1]),
 array([0, 0, 1, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 1]),
 array([0, 0, 1, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 1]),
 array([0, 1, 0, 0, 1, 0]),
 array([0, 1, 0, 1, 0, 0]),
 array([0, 1, 0, 1, 0, 1]),
 array([1, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 1, 0]),
 array([1, 0, 0, 1, 0, 0]),
 array([1, 0, 0, 1, 0, 1]),
 array([1, 0, 1, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 1]),
 array([1, 0, 1, 0, 1, 0])]